In [1]:
# default_exp grids

In [2]:
# hide
# no_test
! [ -e /content ] && pip install -Uqq git+https://github.com/thinkingmachines/geowrangler.git

In [3]:
# hide
# no_test
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# hide
# no_test
# conditionally load nb_black cell magic
import sys

if "google.colab" not in sys.modules:
    from IPython import get_ipython

    ipython = get_ipython()
    ipython.magic("reload_ext lab_black")

# Grids

> grid generators


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thinkingmachines/geowrangler/blob/master/notebooks/00_grids.ipynb)


In [4]:
# exporti
import logging

import numpy as np
from fastcore.basics import patch
from geopandas import GeoDataFrame
from shapely.geometry import Polygon

logger = logging.getLogger(__name__)

In [5]:
# export


class SquareGridGenerator:
    def __init__(
        self,
        gdf: GeoDataFrame,  # a geodataframe to create grids from
        cell_size: float,  # height and width of a square cell in meters
        grid_projection: str = "EPSG:3857",  # projection of grid output
    ):
        self.gdf = gdf
        self.cell_size = cell_size
        self.grid_projection = grid_projection

In [6]:
# export


@patch
def get_ranges(self: SquareGridGenerator) -> (float, float):
    """Get ranges of the top and left corners of grid"""
    reprojected_gdf = self.gdf.to_crs(self.grid_projection)
    minx, miny, maxx, maxy = reprojected_gdf.total_bounds
    xrange = np.arange(minx, maxx, self.cell_size)
    yrange = np.arange(miny, maxy, self.cell_size)
    return xrange, yrange

In [7]:
# export


@patch
def create_cell(
    self: SquareGridGenerator,
    x: float,  # x coord of top left
    y: float,  # y coord of top left
) -> Polygon:
    """Create a square cell based on the top left coordinates and cell_size"""
    return Polygon(
        [
            (x, y),
            (x + self.cell_size, y),
            (x + self.cell_size, y + self.cell_size),
            (x, y + self.cell_size),
        ]
    )

In [8]:
# export


@patch
def generate_grid(self: SquareGridGenerator) -> GeoDataFrame:
    xrange, yrange = self.get_ranges()
    polygons = []
    for x_idx, x in enumerate(xrange):
        for y_idx, y in enumerate(yrange):
            polygons.append(
                {"x": x_idx, "y": y_idx, "geometry": self.create_cell(x, y)}
            )
    dest = GeoDataFrame(polygons, geometry="geometry", crs=self.grid_projection)
    dest_reproject = dest.to_crs(self.gdf.crs)
    final = dest_reproject[dest_reproject.intersects(self.gdf.unary_union)]
    return final

In [1]:
# hide
# no_test
from nbdev.export import notebook2script

notebook2script("00_grids.ipynb")

Converted 00_grids.ipynb.
